In [1]:
"""
Temporary routine for generation of datasets
for demonstration purposes

It is a "Fake Data Mediator" for use with the first
version of the Interactive Topic Model Trainer
"""

import argparse
import json
import os
from pathlib import Path
from langdetect import detect
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [32]:
#This variables will be passed as parameters via command line

parquet_table = '/export/ml4ds/IntelComp/Datalake/SemanticScholar/20220201/papers.parquet'
selectFields = 'title, paperAbstract, doi, year, fieldsOfStudy'
filterCondition = "array_contains(fieldsOfStudy, 'Computer Science')"
path_dataset = "/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/fromHDFS_scalability/S2CS_1.parquet"
perc = float(1)/100

#parquet_table = '/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet'
#selectFields = 'title, objective, startDate, ecMaxContribution, euroSciVocCode'
#filterCondition = ''
#path_dataset = "/export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/fromHDFS/CORDIS.parquet"

In [33]:
# We read the table with the output of NLP processes and identify id field 
lemmas_table = parquet_table.split('.parquet')[0] + '_NLP.parquet'
lemmas_df = spark.sql(f"SELECT * FROM parquet.`{lemmas_table}`")
# Obtain the name of the column that has to be used as the main identifier for the corpus
id_fld = [el for el in lemmas_df.columns if el in ['projectID', 'id', 'appln_id']][0]

#print('Number of documents in dataset:', lemmas_df.count())
#lemmas_df.show(n=2, truncate=120, vertical=True)

22/08/20 00:35:53 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [34]:
# We read the main table including selected fields and the identifier
flds = [el.strip() for el in selectFields.split(',')]
query = "SELECT " + id_fld + " AS id, " + (",").join(flds) + \
                " FROM parquet.`" + parquet_table + "`"

# Add filtering condition to SELECT clause if necessary
if len(filterCondition.strip()):
    query += " WHERE " + filterCondition
dataset = spark.sql(query)

#print('Number of documents in dataset:', dataset.count())
#dataset.show(n=2, truncate=120, vertical=True)

22/08/20 00:35:53 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [35]:
# Join tables
lemmas_df = lemmas_df.withColumnRenamed(id_fld,"id")
dataset = (dataset.join(lemmas_df, dataset.id ==  lemmas_df.id, "left")
                      .drop(lemmas_df.id)
                )

#print('Number of documents in dataset:', dataset.count())
#dataset.show(n=10, truncate=120, vertical=True)

In [36]:
# Sampling dataset if necessary
if perc < 1:
    dataset = dataset.sample(fraction=perc)

print('Number of documents in dataset:', dataset.count())
#dataset.show(n=10, truncate=120, vertical=True)

Number of documents in dataset: 148339


In [37]:
# Save dataset
dataset.write.parquet(f"file://{path_dataset}",
    mode="overwrite",
)

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/opt/spark-3.1.1-bin-2.8.3/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark-3.1.1-bin-2.8.3/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/spark-3.1.1-bin-2.8.3/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
/opt/spark-3.1.1-bin-2.8.3/python/pyspark/context.py:460: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed 